# Lecture 5: Decorators

> Assigning Functions to Variables

> Defining Functions Inside other Functions

> Passing Functions as Arguments to other Functions

> Functions Returning other Functions

> Nested Functions have access to the Enclosing Function's Variable Scope

> Creating Decorators

> Applying Multiple Decorators to a Single Function

> Accepting Arguments in Decorator Functions

> Defining General Purpose Decorators

> Passing Arguments to the Decorator

> Debugging Decorators

> Decorator: @lru_cache()

> Decorator: @cached_property()

> Python Decorators Summary

> Example: Measure performance of a function

> References

In [ ]:
"""Decorator:
A decorator is a design pattern in Python that allows a user to add new functionality 
to an existing object without modifying its structure.

Decorators are usually called before the definition of a function you want to decorate.
"""

### Assigning Functions to Variables

In [ ]:
def plus_one(number):
    return number + 1

add_one = plus_one
add_one(5) # => plus_one(5)

### Defining Functions Inside other Functions

In [ ]:
def plus_one(number):
    def add_one(number):
        return number + 1
    
    result = add_one(number)
    return result

plus_one(4)

### Passing Functions as Arguments to other Functions

Functions can also be passed as parameters to other functions.

In [ ]:
def plus_one(number):
    return number + 1

def function_call(function):
    number_to_add = 5
    return function(number_to_add)

function_call(plus_one)

### Functions Returning other Functions

A function can also generate another function.

In [ ]:
def hello_function():
    def say_hi():
        return "Hi"
    
    return say_hi

hello = hello_function()
hello()

In [ ]:
# hello_function()()

### Nested Functions have access to the Enclosing Function's Variable Scope

Python allows a nested function to access the outer scope of the enclosing function. 

This is a critical concept in decorators - this pattern is known as a Closure.

In [ ]:
def print_message(message):
    "Enclosong Function"
    def message_sender():
        "Nested Function"
        print(message)
    
    message_sender()

print_message("Some random message")

### Creating Decorators

In [ ]:
def uppercase_decorator(function):
    def wrapper():
        func = function()
        make_uppercase = func.upper()
        return make_uppercase
    return wrapper

In [ ]:
def say_hi():
    return 'hello there'

decorate = uppercase_decorator(say_hi)
decorate()

In [ ]:
"""
However, Python provides a much easier way for us to apply decorators.
We simply use the '@' symbol before the function we'd like to decorate.
"""

In [ ]:
@uppercase_decorator
def say_hi():
    return 'hello there'

say_hi()

### Applying Multiple Decorators to a Single Function

We can use multiple decorators to a single function.

However, the decorators will be applied in the order that we've called them. 

In [ ]:
def split_string(function):
    def wrapper():
        func = function()
        splitted_string = func.split()
        return splitted_string
    return wrapper

In [ ]:
@split_string
@uppercase_decorator
def say_hi():
    return 'hello there'

say_hi()

In [ ]:
# @uppercase_decorator
# @split_string
# def say_hi():
#     return 'hello there'

# say_hi()

### Accepting Arguments in Decorator Functions

Sometimes we might need to define a decorator that accepts arguments. 

We achieve this by passing the arguments to the wrapper function.

In [ ]:
def decorator_with_arguments(function):
    def wrapper_accepting_arguments(arg1, arg2):
        print(f"Arguments: {arg1}, {arg2}")
        function(arg1, arg2)
    return wrapper_accepting_arguments


@decorator_with_arguments
def cities(city_one, city_two, city_three="Perm"):
    print(f"Cities I love are {city_one} and {city_two}")

cities("Moscow", "Saint Petersburg")

### Defining General Purpose Decorators

To define a general purpose decorator that can be applied to any function we use *args and **kwargs.

*args and **kwargs collect all positional and keyword arguments and stores them in the args and kwargs variables. 

*args and **kwargs allow us to pass as many arguments as we would like during function calls.

In [ ]:
def a_decorator_passing_arbitrary_arguments(function_to_decorate):
    def a_wrapper_accepting_arbitrary_arguments(*args, **kwargs):
        print(f'The positional arguments are {args}')
        print(f'The keyword arguments are {kwargs}')
        function_to_decorate(*args)
    return a_wrapper_accepting_arbitrary_arguments

In [ ]:
@a_decorator_passing_arbitrary_arguments
def function_with_no_argument():
    print("No arguments here.")

function_with_no_argument()

In [ ]:
# the decorator using positional arguments

@a_decorator_passing_arbitrary_arguments
def function_with_arguments(a, b, c):
    print(a, b, c)

function_with_arguments(1, 2, 3)

In [ ]:
# the decorator using keyword arguments

@a_decorator_passing_arbitrary_arguments
def function_with_keyword_arguments():
    print("This has shown keyword arguments")

function_with_keyword_arguments(first_name="Artur", last_name="Saakyan")

### Passing Arguments to the Decorator

We define a decorator maker that accepts arguments then define a decorator inside it.

We then define a wrapper function inside the decorator as we did earlier.

In [19]:
def decorator_maker_with_arguments(decorator_arg1, decorator_arg2="Numpy", decorator_arg3="Scikit-Learn"):
    def decorator(func):
        def wrapper(function_arg1, function_arg2, function_arg3):
            "This is the wrapper function"
            print("The wrapper can access all the variables\n"
                  "\t- from the decorator maker: {0} {1} {2}\n"
                  "\t- from the function call: {3} {4} {5}\n"
                  "and pass them to the decorated function"
                  .format(decorator_arg1, decorator_arg2, decorator_arg3,
                          function_arg1, function_arg2, function_arg3))
            
            return func(function_arg1, function_arg2, function_arg3)

        return wrapper

    return decorator

In [22]:
pandas = "Pandas"

@decorator_maker_with_arguments(pandas, "Numpy", "Scikit-Learn")
def decorated_function_with_arguments(function_arg1, function_arg2, function_arg3):
    print("This is the decorated function and it only knows about its arguments: {0}"
           " {1}" " {2}".format(function_arg1, function_arg2, function_arg3))

decorated_function_with_arguments(pandas, "Science", "Tools")

The wrapper can access all the variables
	- from the decorator maker: Pandas Numpy Scikit-Learn
	- from the function call: Pandas Science Tools
and pass them to the decorated function
This is the decorated function and it only knows about its arguments: Pandas Science Tools


### Debugging Decorators

As we have noticed, decorators wrap functions. 

- the original function name
- its docstring
- parameter list

are all hidden by the wrapper closure.

For example, when we try to access the decorated_function_with_arguments metadata, we'll see the wrapper closure's metadata.

This presents a challenge when debugging.

In [ ]:
dir(decorated_function_with_arguments)[:5]

In [ ]:
decorated_function_with_arguments.__name__

In [ ]:
decorated_function_with_arguments.__doc__

In [ ]:
"""
In order to solve this challenge Python provides a 'functools.wraps' decorator.
This decorator copies the lost metadata from the undecorated function to the decorated closure.
"""

In [ ]:
import functools

def uppercase_decorator(func):
    
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    
    return wrapper

In [ ]:
@uppercase_decorator
def say_hi():
    "This will say hi"
    return 'hello there'

say_hi()

In [ ]:
"""
It is advisable and good practice to always use 'functools.wraps' when defining decorators. 
It will save you a lot of headache in debugging.
"""

say_hi.__name__, say_hi.__doc__

### Decorator:  @lru_cache()

lru_cache() is one such function in functools module which helps in reducing the execution time of the function by using memoization technique.

**Syntax:**

@lru_cache(maxsize=128, typed=False)


**Parameters:**

**maxsize**: This parameter sets the size of the cache, the cache can store upto maxsize most recent function calls, if maxsize is set to None, the LRU feature will be disabled and the cache can grow without any limitations

**typed**: If typed is set to 'True', function arguments of different types will be cached separately. 

For example, f(3) and f(3.0) will be treated as distinct calls with distinct results and they will be stored in two separate entries in the cache

In [ ]:
import time

# Function that computes Fibonacci numbers without lru_cache
def fib_without_cache(n):
    if n < 2:
        return n
    return fib_without_cache(n - 1) + fib_without_cache(n - 2)

start = time.time()
fib_without_cache(30)
end = time.time()

without_cache = end - start

print(f"Time taken to execute the function without lru_cache is {without_cache:.6f} sec")

In [ ]:
# Function that computes Fibonacci numbers with lru_cache

import time
from functools import lru_cache

@lru_cache()
def fib_with_cache(n):
    if n < 2:
        return n
    return fib_with_cache(n - 1) + fib_with_cache(n - 2)

start = time.time()
fib_with_cache(30)
end = time.time()

with_cache = end - start

print(f"Time taken to execute the function with lru_cache is {with_cache:.6f}")

In [ ]:
without_cache // with_cache

### Decorator: @cached_property()

The @cached_property is a decorator which transforms a method of a class into a property whose value is computed only once and then cached as a normal attribute. Therefore, the cached result will be available as long as the instance will persist and we can use that method as an attribute of a class i.e

Writing    : instance.method

Instead of : instance.method()

It is similar to @property(), but @cached_property() comes with an extra feature and that is caching.

**Why caching?**

The cache memory is a high-speed memory available inside CPU in order to speed up access to data and instructions. 

Therefore, the cache is a place that is quick to access. The result can be computed and stored once and from next time, the result can be accessed without recomputing it again. So, it is useful in case of expensive computations.

#### Difference between @property and @cached_property

In [ ]:
# Using @property

class Sample():
    def __init__(self):
        self.result = 50
    
    @property
    def increase(self):
        self.result += 50
        return self.result

In [ ]:
%%time
obj = Sample()
print(obj.increase, obj.increase, obj.increase)

In [ ]:
# Using @cached_property

from functools import cached_property


class Sample():
    def __init__(self):
        self.result = 50
    
    @cached_property
    def increase(self):
        self.result += 50
        return self.result

In [ ]:
%%time
obj = Sample()
print(obj.increase, obj.increase, obj.increase)

### Example: Measure performance of a function

We will use the decorator to test the performance list generation using four methods:

- range
- list comprehension
- append
- concatenation

In [ ]:
import tracemalloc
from functools import wraps
from time import perf_counter


def measure_performance(func):
    '''Measure performance of a function'''
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        tracemalloc.start()
        start_time = perf_counter()
        func(*args, **kwargs)
        current, peak = tracemalloc.get_traced_memory()
        finish_time = perf_counter()
        print(f'Function: {func.__name__}')
        print(f'Method:   {func.__doc__}')
        print(f'Memory usage:        {current / 10**6:.6f} MB \n'
              f'Peak memory usage:   {peak / 10**6:.6f} MB ')
        print(f'Time elapsed:        {finish_time - start_time:.6f} sec')
        print(f'{"-" * 40}')
        tracemalloc.stop()
    
    return wrapper

In [ ]:
@measure_performance
def make_list1():
    '''Range'''
    
    my_list = list(range(100_000))


@measure_performance
def make_list2():
    '''List comprehension'''

    my_list = [l for l in range(100_000)]


@measure_performance
def make_list3():
    '''Append'''

    my_list = []
    for item in range(100_000):
        my_list.append(item)


@measure_performance
def make_list4():
    '''Concatenation'''

    my_list = []
    for item in range(100_000):
        my_list = my_list + [item]

In [ ]:
print(make_list1())
print(make_list2())
print(make_list3())
print(make_list4())

### Python Decorators Summary

Decorators dynamically alter the functionality of a function, method, or class without having to directly use subclasses or change the source code of the function being decorated.

Using decorators in Python also ensures that your code is DRY (Don't Repeat Yourself). 

Decorators have several use cases such as:
- Authorization in Python frameworks such as Flask and Django (is this person authorized or not?!)
- Logging
- Measuring execution time

### References
<ol>
<li> <a href="https://www.datacamp.com/community/tutorials/decorators-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=278443377095&utm_targetid=aud-299261629574:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=9047064&gclid=Cj0KCQjwiNSLBhCPARIsAKNS4_dREIkpGsPXvvQZ97TnbCHlnotEmdpDpAj5KqvdI-A8foCcuso4QgYaAvdhEALw_wcB">Decorators in Python</a> </li>
<li> <a href="https://www.geeksforgeeks.org/python-functools-lru_cache/">Python Functools - @lru_cache()</a> </li>
<li> <a href="https://www.geeksforgeeks.org/python-functools-cached_property/">Python Functools - @cached_property()</a> </li>
</ol>